In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
import os
ROOT_PATH = "/content/drive/MyDrive/project-m3-2024-lasmer"
print(os.listdir(ROOT_PATH)) # Check the content of the path
os.chdir(ROOT_PATH) # cd into directory
print(os.listdir(".")) # Check the content of current folder

['generate.py', 'utils.py', 'README.md', 'rag.ipynb', '_tests', 'models', 'model', 'pdfs', '_templates', '__pycache__', 'datasets', 'evaluator.py', 'wandb', 'results', 'yassinechaouch', 'Mistral-merged-mcq', 'Mistral-merged-mcq-final', 'merged_model', 'requirements.txt', 'full_merged_model', 'train_mistral.ipynb', 'all_pdfs', 'embeddings', 'metrics.json', 'evaluation.ipynb', 'main_config.yaml', 'embeddings.ipynb', 'evaluation2.ipynb']
['generate.py', 'utils.py', 'README.md', 'rag.ipynb', '_tests', 'models', 'model', 'pdfs', '_templates', '__pycache__', 'datasets', 'evaluator.py', 'wandb', 'results', 'yassinechaouch', 'Mistral-merged-mcq', 'Mistral-merged-mcq-final', 'merged_model', 'requirements.txt', 'full_merged_model', 'train_mistral.ipynb', 'all_pdfs', 'embeddings', 'metrics.json', 'evaluation.ipynb', 'main_config.yaml', 'embeddings.ipynb', 'evaluation2.ipynb']


In [3]:
! pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.4/418.4 kB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [6]:
import re
import os
import json
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import fitz  # PyMuPDF
from tqdm import tqdm

def preprocess_text(text):
    return text.strip()

def process_pdf(file_path):
    # Open the PDF file
    document = fitz.open(file_path)

    whole_text = ''

    # Extract the text from the PDF
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        page_content = preprocess_text(page.get_text("text").replace('\n', ' ').replace('\uf0b7', ' '))
        whole_text += page_content + " "

    # Split text into chunks of approximately 800 words with overlap
    words = whole_text.split()
    chunk_size = 200
    chunk_overlap = 50
    chunks = []

    for i in range(0, len(words), chunk_size - chunk_overlap):
        chunk = words[i:i + chunk_size]
        chunks.append(" ".join(chunk))

    return chunks

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def embed_text_chunks(chunks, model, tokenizer):
    embeddings = []
    for chunk in chunks:
        encoded_input = tokenizer(chunk, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            model_output = model(**encoded_input)
        chunk_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
        chunk_embedding = F.normalize(chunk_embedding, p=2, dim=1)
        embeddings.append(chunk_embedding.squeeze().numpy().tolist())
    return embeddings

def save_embeddings_and_chunks(embeddings, chunks, output_path):
    data = {'embeddings': embeddings, 'chunks': chunks}
    with open(output_path, 'w') as f:
        json.dump(data, f)

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:

# Directory containing PDFs
pdf_directory = "all_pdfs"

# Output directory for embeddings
output_directory = "embeddings"
os.makedirs(output_directory, exist_ok=True)

all_embeddings = []
all_chunks = []

for pdf_file in tqdm(os.listdir(pdf_directory), desc="Processing PDFs"):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, pdf_file)
        chunks = process_pdf(pdf_path)
        embeddings = embed_text_chunks(chunks, model, tokenizer)
        all_chunks.extend(chunks)
        all_embeddings.extend(embeddings)

# Save all embeddings and chunks to a single JSON file
output_path = os.path.join(output_directory, "all_pdfs_embeddings.json")
save_embeddings_and_chunks(all_embeddings, all_chunks, output_path)

print("Embeddings and text chunks for all PDFs generated and saved successfully.")

# code to retrieve

In [7]:
import json
import torch
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity

def embed_prompt(prompt, model, tokenizer):
    encoded_input = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        model_output = model(**encoded_input)
    prompt_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
    prompt_embedding = F.normalize(prompt_embedding, p=2, dim=1)
    return prompt_embedding.squeeze().numpy()

def retrieve_top_chunks(prompt_embedding, embeddings, chunks, top_k=3):
    similarities = cosine_similarity([prompt_embedding], embeddings)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [chunks[i] for i in top_indices], [similarities[i] for i in top_indices]

# Load embeddings and chunks from JSON file
output_path = "embeddings/all_pdfs_embeddings.json"
with open(output_path, 'r') as f:
    data = json.load(f)
    all_embeddings = data['embeddings']
    all_chunks = data['chunks']

# Embed the prompt and retrieve top matching chunks
prompt = "how to delete an element from a linked list ? "
prompt_embedding = embed_prompt(prompt, model, tokenizer)
top_chunks, top_similarities = retrieve_top_chunks(prompt_embedding, all_embeddings, all_chunks)

print("Top matching chunks:")
for i, (chunk, similarity) in enumerate(zip(top_chunks, top_similarities)):
    print(f"Rank {i+1}, Similarity: {similarity:.4f}")
    print(chunk)
    print()


Top matching chunks:
Rank 1, Similarity: 0.7841
the “first” element. Deleting a Node from a Linked List After adding a node to a linked list, deleting a node from a list is the next most important operation.To delete a node from a linked list, use list_del(): list_del(struct list_head *entry) This function removes the element entry from the list. Note that it does not free any memory belonging to entry or the data structure in which it is embedded; this function merely removes the element from the list.After calling this, you would typically destroy your data structure and the list_head inside it. For example, to delete the fox node we previous added to fox_list: list_del(&f->list); Note the function does not receive as input fox_list. It simply receives a specific node and modifies the pointers of the previous and subsequent nodes such that the given node is no longer part of the list.The implementation is instructive: static inline void __list_del(struct list_head *prev, struct list_